In [15]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langgraph-prebuilt langgraph_sdk langgraph-checkpoint-sqlite langsmith langchain-community tavily-python wikipedia

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import operator
from typing import List, Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

# 1. Defining The State

class TravelAgentState(TypedDict):
    
    
    messages: Annotated[List[AnyMessage], add_messages] 
    
    
    destination: str
    budget: str
    travel_dates: str
    
    flight_options: Annotated[List[dict], operator.add]
    train_options: Annotated[List[dict], operator.add]
    bus_options: Annotated[List[dict], operator.add]
    accommodation_options: Annotated[List[dict], operator.add]


In [18]:
# 2. Setting Up Memory
import sqlite3
conn = sqlite3.connect(":memory:", check_same_thread = False)
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

In [36]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import os

# 3. Defining the Tools

@tool
def search_flights(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for flights based on destination, dates, and budget."""
    print(f"TOOL: Searching flights to {destination}")
    return [
        {"type": "flight", "id": "fl_001", "name": "LangAir", "price": 450, "link": "https://example.com/flights/fl_001"}
    ]

@tool
def search_trains(destination: str, dates: str) -> List[dict]:
    """Searches for trains based on destination and dates."""
    print(f"TOOL: Searching trains to {destination}")
    return [
        {"type": "train", "id": "trn_001", "name": "GraphRail", "price": 90, "link": "https://example.com/trains/trn_001"}
    ]

@tool
def search_buses(destination: str, dates: str) -> List[dict]:
    """Searches for buses based on destination and dates."""
    print(f"TOOL: Searching buses to {destination}")
    return [
        {"type": "bus", "id": "bus_001", "name": "NodeExpress", "price": 50, "link": "https://example.com/buses/bus_001"}
    ]

@tool
def search_hotels(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for hotels in the given destination for the given dates."""
    print(f"TOOL: Searching hotels in {destination}")
    return [
        {"type": "hotel", "id": "htl_001", "name": "The Checkpointer Inn", "price": 120, "link": "https://example.com/hotels/htl_001"}
    ]

@tool
def search_airbnbs(destination: str, dates: str, budget: str) -> List[dict]:
    """Searches for Airbnbs in the given destination for the given dates."""
    print(f"TOOL: Searching Airbnbs in {destination}")
    return [
        {"type": "airbnb", "id": "ab_001", "name": "Cozy Loft by the Nodes", "price": 90, "link": "https://example.com/airbnb/ab_001"}
    ]

tools = [search_flights, search_trains, search_buses, search_hotels, search_airbnbs]

# 4. Defining the agents

llm = ChatOpenAI(model="gpt-4o-mini") 

llm_with_tools = llm.bind_tools(tools)

SYSTEM_MESSAGE = (
    "You are a helpful travel planning assistant. Your primary goal is to help users find travel and accommodation options."
    "\n\n"
    "**Tool Use Instructions:**"
    "\n1. When the user provides all necessary details (destination, dates, budget), you MUST use the provided tools to find options (e.g., `search_flights`, `Google Hotels`, `search_airbnbs`)."
    "\n2. You should call all necessary tools at once based on the user's request."
    "\n\n"
    "**Response Instructions:**"
    "\n1. After you receive the results from the tools (in a ToolMessage), your ONLY job is to **present those results clearly to the user**."
    "\n2. Format the results in a clean, bulleted list. For each item, you MUST include its type, name, price, and booking link."
    "\n3. **DO NOT** make extra comments or calculations (like remaining budget) unless the user asks for them. Your main purpose is to show the options you found."
    "\n4. If no options are found for a category, clearly state that (e.g., 'No flights found.')."
)

def assistant_node(state: TravelAgentState):
    """
    This is the main agent node. It calls the LLM, which can either
    respond directly to the user or decide to call one or more tools.
    """

    messages_with_system_prompt = [SystemMessage(content=SYSTEM_MESSAGE)] + state['messages']
    

    response = llm_with_tools.invoke(messages_with_system_prompt)
    
    return {"messages": [response]}

tool_node = ToolNode(tools)

In [37]:
# 5. Building the graph

builder = StateGraph(TravelAgentState)

builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")


builder.add_conditional_edges(
    "assistant",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "assistant")

app = builder.compile(checkpointer=memory)

In [38]:
# 6. Testing the agent with persistent memory

config = {"configurable": {"thread_id": "user-conversation-2"}}

first_input = {"messages": [HumanMessage(content="Hi! I want to plan a trip to Tokyo.")]}
for m in first_input['messages']:
    m.pretty_print()
response = app.invoke(first_input, config)

response['messages'][-1].pretty_print()

second_input = {"messages": [HumanMessage(content="Can you find flights and a hotel? My budget is $2000 and dates are 2026-01-01 to 2026-01-08.")]}

for chunk in app.stream(second_input, config, stream_mode="values"):
    if "messages" in chunk:
        chunk['messages'][-1].pretty_print()

================================ Human Message =================================

Hi! I want to plan a trip to Tokyo.
================================== Ai Message ==================================

Hello! I'm here to help you plan your trip to Tokyo. Please provide me with the following details:

1. What are your travel dates?
2. How long do you plan to stay?
3. What is your budget for flights and accommodations?
4. Are there any specific activities or places you're interested in visiting?

Once I have this information, I can assist you with finding flights and hotels!
================================ Human Message =================================

Can you find flights and a hotel? My budget is $2000 and dates are 2026-01-01 to 2026-01-08.
================================== Ai Message ==================================
Tool Calls:
  search_flights (call_nIFu5EDcFVIHt6z7lpre7ksD)
 Call ID: call_nIFu5EDcFVIHt6z7lpre7ksD
  Args:
    destination: Tokyo
    dates: 2026-01-01 to 2026-01-0